In [1]:
import ipympl
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas

%matplotlib widget


In [2]:
kosten_df_zonder_geo = pd.read_csv('kosten_df.csv')
gemeenten = geopandas.read_file('gemeentegrenzen_2019_vektis_namen.shp')
gemeenten = gemeenten.rename(columns={'gemeentena': 'gemeentenaam'})


In [3]:
kosten_df = kosten_df_zonder_geo.merge(gemeenten, on='gemeentenaam', how='left')
kosten_df = geopandas.GeoDataFrame(kosten_df, crs={'init': 'epsg:3857'}, geometry=kosten_df.geometry)

# Grensoverschrijdende zorgkosten:
## Kies populatie, jaar en uitkomstmaat

In [5]:
# see user_defined scheme
def plot_map(populatiefilter=[('Hele populatie', 'hele_populatie'),
                              ('0 t/m 19 jaar', '0tm19'),
                              ('20 t/m 64 jaar', '20tm64'),
                              ('65 jaar en ouder', '65plus')],
             plotjaar=widgets.IntSlider(min=2011, max=2017, step=1, value=2011),
             uitkomstmaat=[('XBC Kosten per inwoner van gemeente', 'go_kosten_per_inwoner_gemeente'),
                           ('XBC gemeente als perc van totale ZVW gemeente', 'go_kosten_als_perc_van_totale_kosten_gemeente'),
                           ('XCB gemeente als perc van XBC landelijk', 'go_kosten_als_perc_van_go_landelijk')]):
    
    subdf = kosten_df.loc[(kosten_df.jaar == plotjaar) & (kosten_df.populatie == populatiefilter), ]
    
    # color-scheme (mediaan, mediaan-95%, 95%-99%, 99% en hoger)
    bins = [kosten_df[uitkomstmaat].quantile(q) for q in [0.5, 0.95, 0.99]]
    
    if uitkomstmaat == 'go_kosten_per_inwoner_gemeente':
        legend_title = 'EUR'
    else:
        legend_title = 'Percentage (schaal 1-100)'
        
    ax = subdf.plot(figsize=(10, 6), legend=True, legend_kwds={'loc': 'upper left', 'title': legend_title}, column=uitkomstmaat, scheme='user_defined', classification_kwds={'bins': bins})
    ax.set_title(f'De vier kleuren zijn 0-mediaan, mediaan-95%, 95%-99%, 99% en hoger voor de uitkomstmaat over alle jaren heen')
    ax.set_axis_off();

widgets.interact(plot_map);

interactive(children=(Dropdown(description='populatiefilter', options=(('Hele populatie', 'hele_populatie'), (…